In [36]:
%matplotlib inline
import pandas as pd
import datetime as dt
import random
import itertools
import pdb
import time
from dateutil.parser import parse
from experiment.models import Models, Parameters
from experiment.models import get_metadata_file, get_pubdate_index

mt = get_metadata_file()
p = Parameters()
p.q = "BP"

start_time = time.time()
results = Models.get_results(p)
print "query took {}".format(start_time - time.time())

start_time = time.time()
facets, aliases = Models.get_facets(p, results, 3)
print "facets took {}".format(start_time - time.time())

metadata = [mt[r] for r in results]
q_pubdates = [parse(h["pubdate"]) for h in metadata]

pub_index = get_pubdate_index()

def make_dataframe(p, facets, results):
    start_time = time.time()
    df = pd.DataFrame()
    df['id'] = results
    df['pd'] = q_pubdates
    df[p.q] = [1 for i in q_pubdates]
    for facet in facets:
        alias = [pub_index[a] for a in aliases[facet]]
        facet_dates = set([i for i in set([i for i in itertools.chain(*alias)]).union(pub_index[facet])])
        gus = [int(p) for p in [p in facet_dates for p in q_pubdates]]
        df[facet] = gus
    print "dataframe took {}".format(start_time - time.time())
    return df

df = make_dataframe(p, facets, results)
df = df.groupby([df['pd'].map(lambda x: x.year)]).sum().unstack(0).fillna(0)

for f in facets:
    print list(df[f])

print df[p.q]

DEBUG:experiment:got facets. time to filter


query took -1.53921103477
facets took -1.60714292526
dataframe took -0.00365400314331
[0, 0, 0, 2, 1, 0]
[1, 1, 0, 0, 0, 3]
[0, 0, 0, 2, 0, 2]
pd
2010    10
2011     3
2012     3
2013    10
2014     5
2015     6
dtype: int64


In [57]:
[i for i in df[p.q].axes[0]]

[2010, 2011, 2012, 2013, 2014, 2015]